## BCB546X Python Project

In [6]:
from Bio import SeqIO
from Bio.Data import CodonTable
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
%matplotlib inline

In [17]:
def get_sequences_from_file(fasta_fn): #(1) Defines a function called 'get_sequences_from_file' which takes the name of a fasta file as its argument
    sequence_data_dict = {} #(2) Initializes a dictionary named 'sequence_data_dict'
    for record in SeqIO.parse(fasta_fn, "fasta"):  #(3) Initializes a for-loop over the 'SeqRecord' objects (each consisting of a description line and a sequence) returned iteratively by the SeqIO.parse function operating on the fasta file specified in (1)
        description_list = record.description.split() #(4) Splits the description line from the SeqRecord into a list in which each word from the description line is a separate list item
        species_name = description[1] + " " + description[2]  #(5) Pulls the first two items (words) from the 'description_list' created in (4), separates them with a space, and names this string 'species_name'
        sequence_data_dict[species_name] = record.seq  #(6) Creates an entry in the dictionary with the 'species_name' from (5) as the key and the sequence portion of the 'SeqRecord' as the value
    return(sequence_data_dict)  #(7) Returns the above-generated dictionary as the output of this new function

In [18]:
cytb_seqs = get_sequences_from_file("bears_cytb.fasta")

In [194]:
def test_sequences_from_file(fasta_fn):   #defines a function called "get_sequences_from_file," which will take a fasta file name as its argument
    for record in SeqIO.parse(fasta_fn, "fasta"):  #initializes for-loop over the 'SeqRecord' objects (containing a description line and a sequence) produced by the SeqIO.parse function operating on the fasta file from line (1)
        description_list = record.description.split() #splits the description line for a 'SeqRecord' into a list (named "description") in which each word from the description line becomes a list item
        species_name = description_list[1] + " " + description_list[2]  #pulls the first two items (words) from the "description" list above, separated by a space, and sets this equal to the variable "species_name"
    return(species_name)  #returns the above-generated dictionary as the output of this new function

In [196]:
type(test_sequences_from_file("bears_cytb.fasta"))

str

In [19]:
cytb_seqs

{'Ursus spelaeus': Seq('ATGACCAACATCCGAAAAACCCATCCATTAGCTAAAATCATCAACAACTCATTT...AGA', SingleLetterAlphabet()),
 'Ursus arctos': Seq('ATGACCAACATCCGAAAAACCCACCCATTAGCTAAAATCATCAACAACTCACTT...AGA', SingleLetterAlphabet()),
 'Ursus thibetanus': Seq('ATGACCAACATCCGAAAAACCCATCCATTAGCCAAAATCATCAACAACTCACTC...AGA', SingleLetterAlphabet()),
 'Melursus ursinus': Seq('ATGACCAACATCCGAAAAACCCACCCATTAGCTAAAATCATTAACAACTCACTC...AGA', SingleLetterAlphabet()),
 'Ursus americanus': Seq('ATGACCAACATCCGAAAAACCCACCCATTAGCTAAAATCATCAACAACTCACTT...AGA', SingleLetterAlphabet()),
 'Helarctos malayanus': Seq('ATGACCAACATCCGAAAAACCCACCCATTAGCTAAAATCATTAACAACTCACTT...AGA', SingleLetterAlphabet()),
 'Ailuropoda melanoleuca': Seq('ATGATCAACATCCGAAAAACTCATCCATTAGTTAAAATTATCAACAACTCATTC...AGA', SingleLetterAlphabet()),
 'Tremarctos ornatus': Seq('ATGACCAACATCCGAAAAACTCACCCACTAGCTAAAATCATCAACAGCTCATTC...AGA', SingleLetterAlphabet()),
 'Ursus maritimus': Seq('ATGACCAACATCCGAAAAACCCACCCATTAGCTAAAATCATCAACAACTCATTT...A

In [58]:
bears_df = pd.read_csv("bears_mass.csv")

In [59]:
species_list = list(bears_df.species)

In [57]:
bears_df

,species,mass
0,Ursus spelaeus,550.800
1,Ursus arctos,203.500
2,Ursus thibetanus,99.714
3,Melursus ursinus,100.030
4,Ursus americanus,110.560
5,Helarctos malayanus,47.020
6,Ailuropoda melanoleuca,118.200
7,Tremarctos ornatus,140.700
8,Ursus maritimus,425.100


In [127]:
def translate_function(NucSeq): 
     mito_table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"] # this should work using BioPython (be sure to check what this returns)
     for -loop through every 3rd position in string_nucleotides to get the codon using range subsets
#         # IMPORTANT: if the sequence has a stop codon at the end, you should leave it off
#         # this is how you can retrieve the amino acid: mito_table.forward_table[codon]
#         add the aa to aa_seq_string
#     return(aa_seq_string)

In [62]:
mito_table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"]

In [63]:
mito_table

In [64]:
print(mito_table)

Table 2 Vertebrate Mitochondrial, SGC1

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA W   | A
T | TTG L   | TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L   | CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I(s)| ACT T   | AAT N   | AGT S   | T
A | ATC I(s)| ACC T   | AAC N   | AGC S   | C
A | ATA M(s)| ACA T   | AAA K   | AGA Stop| A
A | ATG M(s)| ACG T   | AAG K   | AGG Stop| G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V(s)| GCG A   | GAG E   | GGG G   

In [67]:
CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"]

In [68]:
print(CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"])

Table 2 Vertebrate Mitochondrial, SGC1

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA W   | A
T | TTG L   | TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L   | CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I(s)| ACT T   | AAT N   | AGT S   | T
A | ATC I(s)| ACC T   | AAC N   | AGC S   | C
A | ATA M(s)| ACA T   | AAA K   | AGA Stop| A
A | ATG M(s)| ACG T   | AAG K   | AGG Stop| G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V(s)| GCG A   | GAG E   | GGG G   

In [103]:
mito_table.forward_table["AGA"]

KeyError: 'AGA'

In [89]:
NucSeq = cytb_seqs[species_list[0]]

In [79]:
type(cytb_seqs[species_list[0]])

Bio.Seq.Seq

In [94]:
cytb_seqs[species_list[0]]

Seq('ATGACCAACATCCGAAAAACCCATCCATTAGCTAAAATCATCAACAACTCATTT...AGA', SingleLetterAlphabet())

In [128]:
aa_seq = ""
for x,y,z in zip(NucSeq[0::3],NucSeq[1::3],NucSeq[2::3]):
    codon = x + y + z
    if codon in ["TAA","TAG","AGA","AGG"]:
        break
    else:
        aa_seq += (mito_table.forward_table[codon])
print(aa_seq)

MTNIRKTHPLAKIINNSFIDLPTPSNISAWWNFGSLLGVCLILQILTGLFLAMHYTSDTTTAFSSITHICRDVHYGWVIRYMHANGASMFFICLFMHVGRGLYYGSYLFSETWNIGIILLLTVMATAFMGYVLPWGQMSFWGATVITNLLSAIPYIGTDLVEWIWGGFSVDKATLTRFFAFHFILPFIILALAAVHLLFLHETGSNNPSGIPSDSDKIPFHPYYTIKDILGALLLTLALAALVLFSPDLLGDPDNYTPANPLSTPPHIKPEWYFLFAYAILRFIPNKLGGVLALIFSILILAIISLLHTSKQRGMMFRPLSQCLFWLLVADLLTLTWIGGQPVEHPFIIIGQLASILYFTIPLVLMPIAGIIENNLLKW


In [186]:
def translate_function(nucseq):
    mito_table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"]
    aa_seq = ""
    for x,y,z in zip(nucseq[0::3],nucseq[1::3],nucseq[2::3]):
        codon = x + y + z
        if codon in ["TAA","TAG","AGA","AGG"]:
            break
        else:
            aa_seq += (mito_table.forward_table[codon])
    return(aa_seq)

In [134]:
len(translate_function(cytb_seqs[species_list[0]]))

379

In [188]:
translate_function(cytb_seqs[species_list[1]])

'MTNIRKTHPLAKIINNSLIDLPTPSNISAWWNFGSLLGVCLILQILTGLFLAMHYTPDTTTAFSSVTHICRDVHYGWVIRYVHANGASIFFICLFMHVGRGLYYGSYLFSETWNIGIILLFTIMATAFMGYVLPWGQMSFWGATVITNLLSAIPYIGTDLVEWIWGGFSVDKATLTRFFAFHFILPFIILALAAVHLLFLHETGSNNPSGIPSDSDKIPFHPYYTIKDILGALLLALTLATLVLFSPDLLGDPDNYTPANPLSTPPHIKPEWYFLFAYAILRSIPNKLGGVLALIFSILILAIIPLLHTSKQRGMMFRPLSQCLFWLLVADLLTLTWIGGQPVEHPFIIIGQLASILYFTILLVLMPIAGIIENNLLKW'

In [156]:
translate_function(cytb_seqs[species_list[2]])

'MTNIRKTHPLAKIINNSLIDLPAPSNISAWWNFGSLLGMCLILQILTGLFLAMHYTSDATTAFSSVAHICRDVHYGWIIRYMHANGASMFFICLFMHVGRGLYYGSYLLSETWNIGIILLFTVMATAFMGYVLPWGQMSFWGATVITNLLSAIPYIGTDLVEWIWGGFSVDKATLTRFFAFHFILPFIILALAAVHLLFLHETGSNNPSGIPSDSDKIPFHPYYTIKDALGALLLILALATLVLFSPDLLGDPDNYTPANPLSTPPHIKPEWYFLFAYAILRSIPNKLGGVLALIFSILILAIIPLLHTSKQRGMMFRPLSQCLFWLLVADLLTLTWIGGQPVEHPFIIIGQLASILYFTILLVLMPIAGIIENNLSKW'

In [144]:
for x in range(9):
    print(len(translate_function(cytb_seqs[species_list[x]])))

379
379
379
379
379
379
379
379
379


In [154]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
def compute_molecular_weight(aa_seq):
    return(ProteinAnalysis(aa_seq).molecular_weight())

In [245]:
compute_molecular_weight(translate_function(cytb_seqs[species_list[0]]))

42458.79919999999

In [251]:
from Bio.SeqUtils import molecular_weight
molecular_weight(translate_function(cytb_seqs[species_list[0]]), seq_type='protein', double_stranded=False, circular=False, monoisotopic=False)

42458.79919999999

In [160]:
def gc_content(nucseq):
    gc = 0
    nuc = 0
    for n in nucseq[:]:
        if n in ['G','C']:
            gc += 1
        if n in ['A','T','C','G']:
            nuc += 1
    gc_ratio = gc / nuc
    return(gc_ratio)

In [240]:
from Bio.SeqUtils import GC
GC(cytb_seqs[species_list[0]])

43.771929824561404

In [161]:
gc_content(cytb_seqs[species_list[0]])

0.437719298245614

In [162]:
gc_content(cytb_seqs[species_list[1]])

0.437719298245614

In [163]:
for x in range(9):
    print(gc_content(cytb_seqs[species_list[x]]))

0.437719298245614
0.437719298245614
0.45614035087719296
0.4517543859649123
0.4394736842105263
0.44298245614035087
0.40789473684210525
0.443859649122807
0.44298245614035087


In [170]:
from Bio.Seq import Seq
def alt_translator(nucseq):
    
coding_dna = cytb_seqs[species_list[0]]
checkseq = coding_dna.translate(table=2, to_stop=True)

In [166]:
cytb_seqs[species_list[0]]

Seq('ATGACCAACATCCGAAAAACCCATCCATTAGCTAAAATCATCAACAACTCATTT...AGA', SingleLetterAlphabet())

In [174]:
same = 0
for x,y in zip(checkseq[:],translate_function(cytb_seqs[species_list[0]])):
    if x == y:
        same += 1
print(same)

379


In [190]:
from Bio.Seq import Seq
def alt_translator(nucseq):
    seq = coding_dna.translate(table="Vertebrate Mitochondrial", to_stop=True)
    aa_seq = ""
    for n in seq:
        aa_seq += n
    return(aa_seq)

In [191]:
alt_translator(cytb_seqs[species_list[0]])

'MTNIRKTHPLAKIINNSFIDLPTPSNISAWWNFGSLLGVCLILQILTGLFLAMHYTSDTTTAFSSITHICRDVHYGWVIRYMHANGASMFFICLFMHVGRGLYYGSYLFSETWNIGIILLLTVMATAFMGYVLPWGQMSFWGATVITNLLSAIPYIGTDLVEWIWGGFSVDKATLTRFFAFHFILPFIILALAAVHLLFLHETGSNNPSGIPSDSDKIPFHPYYTIKDILGALLLTLALAALVLFSPDLLGDPDNYTPANPLSTPPHIKPEWYFLFAYAILRFIPNKLGGVLALIFSILILAIISLLHTSKQRGMMFRPLSQCLFWLLVADLLTLTWIGGQPVEHPFIIIGQLASILYFTIPLVLMPIAGIIENNLLKW'

In [204]:
from Bio.SeqUtils import GC
same2 = 0
for x in range(9):
    if GC(cytb_seqs[species_list[x]]) == 100 * gc_content(cytb_seqs[species_list[x]]):
        same2 += 1
print(same2)

7


In [237]:
x = 0

In [238]:
GC(cytb_seqs[species_list[x]])

43.771929824561404

In [239]:
100 * gc_content(cytb_seqs[species_list[x]])

43.771929824561404